In [34]:
import pandas as pd
import numpy as np
import xarray as xr
import statsmodels.api as sm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import warnings
from scipy.stats import linregress
import geopandas as gpd

def calculate_dewpoint(temp, humidity):
    A = 17.27
    B = 237.7
    alpha = ((A * temp) / (B + temp)) + np.log(humidity/100.0)
    return (B * alpha) / (A - alpha)

def process_data(interp_data, region_data):
    for year, df in interp_data.items():
        df = pd.merge(df, region_data, on=["lat", "lon"], how="inner")
        last_column = df.pop(df.columns[-1])
        df.insert(0, last_column.name, last_column)
        df = df.drop(["lat", "lon"], axis=1)
        interp_data[year] = df

def calculate_bins(series):
    series_range = series.max() - series.min()
    return int(series_range / 1)


def region_scaling(pr_dir, tas_dir, hur_dir, start_year, stop_year, region,  method = None):
    years_list = [str(year) for year in range(start_year, stop_year+1)]
    interp_pr, interp_tas, interp_hur, interp_tdew = {}, {}, {}, {}

    for year in years_list: 
        file_name = f"{year}.csv"
        # pr
        file_pr = pd.read_csv(pr_dir + "\\" + file_name)
        file_pr = file_pr[file_pr.lat > -60]
        # tas
        file_tas = pd.read_csv(tas_dir+ "\\" + file_name)
        file_tas = file_tas[file_tas.lat > -60]
        # hur
        file_hur = pd.read_csv(hur_dir+ "\\" + file_name)
        file_hur = file_hur[file_hur.lat > -60]
        interp_pr[year], interp_tas[year], interp_hur[year] = file_pr, file_tas, file_hur 
    
    # Calculate Dewpoint temperature 

    warnings.filterwarnings(action='ignore')

    interp_tdew = {}

    for key in interp_tas.keys():
        tas = interp_tas[key]
        hur = interp_hur[key]
        tdew = pd.DataFrame()
        tdew[["lat", "lon"]] = tas[["lat", "lon"]]
        for day in tas.columns[2:]:
            tdew[day] = calculate_dewpoint(tas[day], hur[day])
        interp_tdew[key] = tdew 

    if region == "ar6":
        grouped = pd.read_csv("D:\Min\Review GCM\Region and Location\Grouping_Region_AR6.csv")[["lat", "lon", "code"]]
    elif region == "srex":
        grouped = pd.read_csv("D:\Min\Review GCM\Region and Location\Grouping_Region_SREX.csv")[["lat", "lon", "code"]]

    # Match with Pr and Tdew dict 
        
    process_data(interp_pr, grouped)
    process_data(interp_tdew, grouped)

    # Combine tdew and Pr 

    pr = pd.concat([df.set_index(["code"]) for df in list(interp_pr.values())], axis=1).reset_index()
    pr_long = pd.melt(pr, id_vars=['code'], var_name='date', value_name='pr')
    tdew  = pd.concat([df.set_index(['code']) for df in list(interp_tdew.values())], axis=1).reset_index()
    tdew_long = pd.melt(tdew, id_vars=['code'], var_name='date', value_name='tdew')
    pr_long["tdew"] = tdew_long['tdew'].values

    # get Wet-day 

    pr_long = pr_long[pr_long.pr > 0.1]
    
    result = {}

    if method == "quantile_regression" or method is None: 

        qr =  pr_long.groupby('code').apply(lambda group: sm.QuantReg(np.log(group['pr']), sm.add_constant(group[['tdew']])).fit(q=0.99))

        dfs = []

        for group_key, group_result in qr.items():
            code = group_key
            slope_coefficient = group_result.params["tdew"]
            df = pd.DataFrame({'code': [code], 'slope_coefficient': slope_coefficient})
            dfs.append(df)
        
        result_qr = pd.concat(dfs, ignore_index=True)

        result_qr["Scaling"] =  100*(np.e**result_qr["slope_coefficient"] - 1)
        result_qr = result_qr[["code", "Scaling"]]

        # Add to result dict 
        result["quantile_regression"] = result_qr

    if method == "binning_data_point" or method is None:
        df = pr_long.copy()
        n_bins = 30
        df["bin"] = df.groupby('code')['tdew'].transform(lambda x: pd.qcut(x, q=n_bins, labels=False, duplicates='drop'))
        dfs = df.drop(['date'], axis = 1)

        bm = dfs.groupby(['code', 'bin']).agg({'pr': lambda x: x.quantile(0.99), 'tdew': 'mean'}).reset_index()
        bm['log_p_99'] = bm['pr'].apply(lambda x: 0 if x <= 0 else 1 if x == 1 else np.log(x))
        bm.columns = ['code', 'bin', 'p_99_pr', 'mean_tdew', 'log_p99']

        slopes = []

        for code, group in bm.groupby('code'):

            slope, _, _, _, _ = linregress(group['mean_tdew'], group['log_p99'])

            slopes.append({'code': code, 'slope': slope})

        result_bm = pd.DataFrame(slopes)

        result_bm["Scaling"] = 100*(np.e**result_bm["slope"] - 1)
        result_bm = result_bm[["code", "Scaling"]]

        # Add to result dict 
        result["binning_equal_data"] = result_bm
        

    if method == "binning_equal_width" or method is None:
        df = pr_long.copy()
        df["bin"] = df.groupby('code')['tdew'].transform(lambda x: pd.cut(x, bins=calculate_bins(x), labels=False, include_lowest=True))

        dfs = df.drop(['date'], axis = 1)

        bm_width =  dfs.groupby(['code', 'bin']).agg({'pr': lambda x: x.quantile(0.99), 'tdew': 'mean'}).reset_index()
        bm_width['log_p_99'] = bm_width['pr'].apply(lambda x: 0 if x <= 0 else 1 if x == 1 else np.log(x))
        bm_width.columns = ['code', 'bin', 'p_99_pr', 'mean_tdew', 'log_p99']

        slopes = []

        for code, group in bm_width.groupby('code'):

            slope, _, _, _, _ = linregress(group['mean_tdew'], group['log_p99'])

            slopes.append({'code': code, 'slope': slope})

        result_bm_width = pd.DataFrame(slopes)

        result_bm_width["Scaling"] = 100*(np.e**result_bm_width["slope"] - 1)

        result_bm_width = result_bm_width[["code", "Scaling"]]

    # Add to result dict 
        
        result["binning_equal_width"] = result_bm_width

    df = pd.concat([df.assign(source=source) for source, df in result.items()])
    # Reset the index
    df.reset_index(drop=True, inplace=True)

    return result, df

In [27]:
year = [y for y in range(2041, 2072)]

In [41]:
acess585 = pd.DataFrame()

for year in range(2041, 2071):
    r,d = region_scaling(pr_dir = "D:\Min\Review GCM\ACCESS-CM2\Interpolated\Pr",
             tas_dir = "D:\Min\Review GCM\ACCESS-CM2\Interpolated\Tas",
             hur_dir = "D:\Min\Review GCM\ACCESS-CM2\Interpolated\Hurs", 
             start_year =  year,
             stop_year = year+20,
             region = "srex")
    d = d.rename(columns = {"Scaling": str(year+20)})
    print("done: ", year)
    if year == 2041: 
        acess585 = d 
    else: acess585 = pd.merge(acess585,d , on = ["code", "source"], how = 'inner')

done:  2041
done:  2042
done:  2043
done:  2044
done:  2045
done:  2046
done:  2047
done:  2048
done:  2049
done:  2050
done:  2051
done:  2052
done:  2053
done:  2054
done:  2055
done:  2056
done:  2057
done:  2058
done:  2059
done:  2060
done:  2061
done:  2062
done:  2063
done:  2064
done:  2065
done:  2066
done:  2067
done:  2068
done:  2069
done:  2070


In [42]:
acess585.to_csv("D:\Min\Review GCM\ACCESS-CM2\Interpolated\\result_585_srex.csv")

In [43]:
acess245 = pd.DataFrame()

for year in range(2041, 2071):
    r,d = region_scaling(pr_dir = "D:\Min\Review GCM\ACCESS-CM2\SSP245\Interpolated\Pr",
             tas_dir = "D:\Min\Review GCM\ACCESS-CM2\SSP245\Interpolated\Tas",
             hur_dir = "D:\Min\Review GCM\ACCESS-CM2\SSP245\Interpolated\Hurs", 
             start_year =  year,
             stop_year = year+20,
             region = "srex")
    d = d.rename(columns = {"Scaling": str(year+20)})
    print("done: ", year)
    if year == 2041: 
        acess245 = d 





        
    else: acess245= pd.merge(acess245,d , on = ["code", "source"], how = 'inner')

done:  2041
done:  2042
done:  2043
done:  2044
done:  2045
done:  2046
done:  2047
done:  2048
done:  2049
done:  2050
done:  2051
done:  2052
done:  2053
done:  2054
done:  2055
done:  2056
done:  2057
done:  2058
done:  2059
done:  2060
done:  2061
done:  2062
done:  2063
done:  2064
done:  2065
done:  2066
done:  2067
done:  2068
done:  2069
done:  2070


In [44]:
acess245.to_csv("D:\Min\Review GCM\ACCESS-CM2\SSP245\Interpolated\\result_245_srex.csv")

In [45]:
nor1 = pd.DataFrame()

for year in range(2041, 2071):
    r,d = region_scaling(pr_dir = "D:\Min\Review GCM\\NorESM2-LM\SSP245\Interpolated\Pr",
             tas_dir = "D:\Min\Review GCM\\NorESM2-LM\SSP245\Interpolated\Tas",
             hur_dir = "D:\Min\Review GCM\\NorESM2-LM\SSP245\Interpolated\Hurs", 
             start_year =  year,
             stop_year = year+20,
             region = "srex")
    d = d.rename(columns = {"Scaling": str(year+20)})
    print("done: ", year)
    if year == 2041: 
        nor1 = d 
    else: nor1= pd.merge(nor1,d , on = ["code", "source"], how = 'inner')

done:  2041
done:  2042
done:  2043
done:  2044
done:  2045
done:  2046
done:  2047
done:  2048
done:  2049
done:  2050
done:  2051
done:  2052
done:  2053
done:  2054
done:  2055
done:  2056
done:  2057
done:  2058
done:  2059
done:  2060
done:  2061
done:  2062
done:  2063
done:  2064
done:  2065
done:  2066
done:  2067
done:  2068
done:  2069
done:  2070


In [49]:
nor1.to_csv("D:\Min\Review GCM\\NorESM2-LM\SSP245\\result_245_srex.csv")

In [47]:
nor2 = pd.DataFrame()

for year in range(2041, 2071):
    r,d = region_scaling(pr_dir = "D:\Min\Review GCM\\NorESM2-LM\SSP585\Interpolated\Pr",
             tas_dir = "D:\Min\Review GCM\NorESM2-LM\\SSP585\Interpolated\Tas",
             hur_dir = "D:\Min\Review GCM\NorESM2-LM\\SSP585\Interpolated\Hurs", 
             start_year =  year,
             stop_year = year+20,
             region = "srex")
    d = d.rename(columns = {"Scaling": str(year+20)})
    print("done: ", year)
    if year == 2041: 
        nor2 = d 
    else: nor2 = pd.merge(nor2,d , on = ["code", "source"], how = 'inner')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 17-18: malformed \N character escape (2792044683.py, line 5)

In [ ]:
nor2.to_csv("D:\Min\Review GCM\\NorESM2-LM\SSP585\\result_585_srex")